In [7]:
import pandas as pd
from datasets import Dataset

# Load Dataset

In [8]:
data_path = [
    "../data/transcripts/transcriptS01E08.csv",
    "../data/transcripts/transcriptS02E09.csv",
    "../data/transcripts/transcriptS03E08.csv",
    "../data/transcripts/transcriptS04E09.csv"
]
transcript_df = pd.concat(
    [pd.read_csv(path, on_bad_lines="skip") for path in data_path],
    ignore_index=True
)
transcript_df.head(10)


,name,line
0,Joyce,"""Let me out of here!"""
1,Joyce,"""Let me out!"""
2,Dr. Brenner,"""Your son."""
3,Dr. Brenner,"""We know you've been in contact with him."""
4,Joyce,"""You have to let me..."""
5,Dr. Brenner,"""When and how did you make contact with him?"""
6,Joyce,"""What?"""
7,Dr. Brenner,"""Hmm?"""
8,Dr. Brenner,"""Six."""
9,Joyce,"""What?"""


In [9]:
transcript_df['number_of_words'] = transcript_df['line'].str.strip().str.split(" ")
transcript_df['number_of_words'] = transcript_df['number_of_words'].apply(lambda x: len(x))
transcript_df.head(10)

,name,line,number_of_words
0,Joyce,"""Let me out of here!""",5
1,Joyce,"""Let me out!""",3
2,Dr. Brenner,"""Your son.""",2
3,Dr. Brenner,"""We know you've been in contact with him.""",8
4,Joyce,"""You have to let me...""",5
5,Dr. Brenner,"""When and how did you make contact with him?""",9
6,Joyce,"""What?""",1
7,Dr. Brenner,"""Hmm?""",1
8,Dr. Brenner,"""Six.""",1
9,Joyce,"""What?""",1


In [10]:
transcript_df['eleven_res'] = 0
transcript_df.loc[(transcript_df['name']=="Eleven")&(transcript_df['number_of_words']>=1),'eleven_res']=1
transcript_df.head()

,name,line,number_of_words,eleven_res
0,Joyce,"""Let me out of here!""",5,0
1,Joyce,"""Let me out!""",3,0
2,Dr. Brenner,"""Your son.""",2,0
3,Dr. Brenner,"""We know you've been in contact with him.""",8,0
4,Joyce,"""You have to let me...""",5,0


In [11]:
indexes_to_take = list(transcript_df[(transcript_df['eleven_res'] == 1)&(transcript_df.index>0)].index)
indexes_to_take[:3]
len(indexes_to_take)

139

In [16]:
system_prompt = """You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_prompt
    prompt += transcript_df.iloc[ind -1]['line'] + '\n'
    prompt += transcript_df.iloc[ind]['line']
    prompts.append(prompt)

In [17]:
print(prompts[3])

You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns 
 "She can make you whatever you like."
 "Eggos?"


In [19]:
df = pd.DataFrame({"prompt": prompts})
df.head()


,prompt
0,"You are Eleven (or El for short), a character ..."
1,"You are Eleven (or El for short), a character ..."
2,"You are Eleven (or El for short), a character ..."
3,"You are Eleven (or El for short), a character ..."
4,"You are Eleven (or El for short), a character ..."


In [20]:
dataset = Dataset.from_pandas(df)